In [1]:
#https://ipyton.org/ipython-docdex/config/extsions/autoreload.html
%reload_ext autoreload
%autoreload 2

#version information
%reload_ext version_information
%version_information Cython, matplotlib, numpy, pandas,  qutip, seaborn, scipy, sklearn, tqdm, version_information,

Software,Version
Python,3.6.4 64bit [GCC 7.2.0]
IPython,6.3.1
OS,Linux 4.9.87 linuxkit aufs x86_64 with debian stretch sid
Cython,0.28.2
matplotlib,2.1.2
numpy,1.14.3
pandas,0.22.0
qutip,The 'qutip' distribution was not found and is required by the application
seaborn,0.8.1
scipy,1.1.0


In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from six.moves import range
from tqdm import tqdm

#some packages for cleaning and wrangling
from collections import defaultdict
import datetime as dt
import re
from tqdm import tqdm

#some packages for natural language processing
import nltk

#some pacakages for working with sentiment
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk.classify import NaiveBayesClassifier
import spacy


from __future__ import print_function

#Only for use in Python 2.6.0a2 and later
from __future__ import print_function
#from nltk.sentiment.util import *
#from nltk.corpus import opinion_lexicon
#from nltk.tokenize import treebank

#some packages for gensim and topic 
from gensim.corpora.dictionary import Dictionary
from itertools import chain
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

print(nltk.__version__)

3.3


In [3]:
! javac -version

javac 1.8.0_45


In [4]:
#downlaod some lexicons from nltk
nltk.download('vader_lexicon')
#nltk.download('punkt')
#nltk.download('wordnet')
nltk.download('stopwords')
#need to download a new tagger 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# load in stop words for english
stop_words = set( nltk.corpus.stopwords.words("english") )

#stop_words.remove("not")  could want for n-gram use later to remove 'not'
# add list of more stop words
extra_stop= ['com', 'twitter', 'pic', 'u', 'http', 'wh', 'many','must',"gov","w","g","#","nd"]
for i in extra_stop:
    stop_words.add(i)

In [6]:
sid = SentimentIntensityAnalyzer()

# some functions

In [7]:
#a function to clean the text of tweets into a new column and to convert the date/time text into one datetime col
def clean_tweet_csv( news_df, tweet_from ):
    """news_df is pandas dataframe from csv for twitter account
    tweet_from is name of twitter accout
    ml_df is the dataframe for doing machine learning in a tidy format"""
    
    new_column_name= '{}_tweet'.format(tweet_from)
    #make column for sentiments
    news_df[new_column_name]=None
    
    
    
    #loop through length of tweets pass
    for i in tqdm ( range(len( news_df ) ) ):
        #note news dataframe has to have text column: replace comma with 
        tweet_combiner=(news_df.text.iloc[i]).replace(',','')
        tweet_combiner=(tweet_combiner).replace('"','')
        tweet_combiner=(tweet_combiner).replace('#','')
        news_df.loc[i,new_column_name] =tweet_combiner
    
        #clean datetime to new column in datetime format
        date=(news_df.loc[i,'date']).replace('"',' ')
        date=(date).replace(',',' ')
        news_df.at[i, "timestamp"] = dt.datetime.strptime(date,"%Y-%m-%d %H:%M")
        
              

In [8]:
def sentiment_vader_tweets( dataframe , tweet_col, sentiment_of):
    '''function to get sentiment for each tweet
    takes 3 inputs: dataframe to add sentiment to , column with tweets
    adds a column to data from sentiment with  pos, neg or nea. Only gives nea if no score for pos or neg'''
    
    new_column_name= 'sentiment_vader_{}'.format(sentiment_of)
    #make column for sentiments
    dataframe[new_column_name]=None
    
    column_with_tweet='{}_news_tweet'.format(sentiment_of)
    
    #loop through length of tweets pass
    for i in range(len( dataframe ) ):
        
        score=0
        try:
            ss=sid.polarity_scores(dataframe.loc[i,tweet_col])
            
            score+=ss['compound']
                
        except AttributeError:
            score+=0
                
        except TypeError: #if None is in fox_tweet column
                score+=0
        

        dataframe.loc[i,new_column_name] =score # if the negative score is higher assign neg     

In [9]:
def token_lem_stop_tweets_col(dataframe):
    """creats new column in dataframe with tokenized, and lemmatized words 
    excluding stop word (not intentional left in)"""
    wnl = WordNetLemmatizer()
  
    dataframe['token_tweet']=None
    dataframe['lemmatized_tweet']=None
    l=len(dataframe)
    for i in tqdm ( range(l) ):
        token_tweets= nltk.word_tokenize( dataframe.clean_tweet[i] )
        token_stop = [ token for token in token_tweets if not token in stop_words ]
        dataframe.token_tweet.iloc[i] = token_stop
        lemmed_tokens= [wnl.lemmatize(token) for token in token_stop]
        lemmed_stop = [ token for token in lemmed_tokens if not token in stop_words ]
        dataframe.lemmatized_tweet.iloc[i] = lemmed_stop

In [ ]:
def news_tweet(newsframe , news_company, ml_df ):
    #get tweet column name
    column_tweet_text='{}_tweet'.format(news_company)
    
    #create column name
    column_name='{}_text_before_tweet'.format(news_company)
    ml_df[column_name]=None
    
    #loop starting at index one on trump dataframe and stopping before last index
    for i in tqdm( range ( len ( ml_df ) -2 ) ): 
        
        #list to hold fox text feed before tweet
        text=[]
        
        # find fox news description between tweets
        t=newsframe[ (newsframe['timestamp'] >= ml_df.loc[i+1, 'timestamp']) & (newsframe['timestamp'] <= ml_df.loc[i, 'timestamp']) ]
        
        t[column_tweet_text].apply(lambda x: text.append(x))
         
        
        ml_df.at[i, column_name] = text
          

In [54]:
def sentiment_vader_news( dataframe , tweet_col, sentiment_of):
    '''function to get sentiment for each tweet
    takes 2 inputs: dataframe to add sentiment to , column with tweets
    adds a column to data from fro sentiment with  pos, neg or nea. Only gives nea if no score for pos or neg'''
    
    new_column_name= 'sentiment_vader_{}'.format(sentiment_of)
    #make column for sentiments
    dataframe[new_column_name]=None
    
    column_with_news_tweet='{}_text_before_tweet'.format(sentiment_of)
    
    #loop through length of tweets pass
    for i in tqdm( range(len( dataframe ) ) ):
        
        score=0
        score_counter=0
        try:
            for j in dataframe.loc[i,column_with_news_tweet]:
                try:
                    ss=sid.polarity_scores(j)
                    score+=ss['compound']
                    score_counter+=0
                except AttributeError:
                    score+=0
                    
        except TypeError: #if None is in {}_tweet column
                score+=0
        

        dataframe.loc[i,new_column_name] =score # if the negative score is higher assign neg
          

In [82]:
def tweet_before_sentiment( newsframe, news_company , ml_df ):
    
    #get tweet column name
    column_tweet_text='sentiment_vader_{}'.format(news_company)
    
    #create column name
    column_name='{}_sentiment_before_tweet'.format(news_company)
    ml_df[column_name]=None
    
    #loop starting at index one on trump dataframe and stopping before last index
    for i in tqdm( range ( len ( ml_df ) -2 ) ): 
        
        #list to hold fox text feed before tweet
        score=[]
        
        # find fox news description between tweets
        t=newsframe[ (newsframe['timestamp'] >= ml_df.loc[i+1, 'timestamp']) & (newsframe['timestamp'] <= ml_df.loc[i, 'timestamp']) ]
         
        
        ml_df.at[i, column_name] = t[column_tweet_text].sum()/t[column_tweet_text].count()
          

# Load the tweet for Trump
The tweets were scrapped using a python code provided by Jeremy Cunningham. See here:
The tweets were saved to a csv file, and loaded here.

In [12]:
#load scrapped tweets
trump_18 = pd.read_csv('trump_current_year.csv')

In [13]:
#clean the data
trump_18.dropna(axis=0,subset=['text'], inplace=True)#drop rows with NAN in text column

trump_18.dropna(axis=0,how='all', inplace=True)#drop rows with NaN in all columns

trump_18=trump_18.reset_index()#reset index to replace dropped indices



In [14]:
#call function to get a cleaned tweet column called 'trump_18_tweet' and a 'timestamp' column
clean_tweet_csv( trump_18, 'trump_18')

100%|██████████| 2415/2415 [00:01<00:00, 1569.16it/s]


In [15]:
#inspect 
trump_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,trump_18_tweet,timestamp
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,https://twitter.com/realDonaldTrump/status/999...,Today it was my great honor to present the Med...,2018-05-24 14:18:00


Now we get the sentiment for each trump tweet

In [20]:
#call function to add sentiment column 
sentiment_vader_tweets(trump_18 , "trump_18_tweet" , 'trump_18')

In [21]:
#inspect again
trump_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,trump_18_tweet,timestamp,sentiment_vader_trump_18
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,https://twitter.com/realDonaldTrump/status/999...,Today it was my great honor to present the Med...,2018-05-24 14:18:00,0.8316


# Lets take a look at the sentiment using Bokeh Plotting

In [26]:
trump_18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2415 entries, 0 to 2414
Data columns (total 14 columns):
index                       2415 non-null int64
username                    2415 non-null object
date                        2415 non-null object
retweets                    2415 non-null int64
favorites                   2415 non-null int64
text                        2415 non-null object
geo                         0 non-null float64
mentions                    402 non-null object
hashtags                    354 non-null object
id                          2415 non-null int64
permalink                   2415 non-null object
trump_18_tweet              2415 non-null object
timestamp                   2415 non-null datetime64[ns]
sentiment_vader_trump_18    2415 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(8)
memory usage: 264.2+ KB


In [28]:
trump_18.sentiment_vader_trump_18= trump_18.sentiment_vader_trump_18.astype("float64")

In [29]:
trump_18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2415 entries, 0 to 2414
Data columns (total 14 columns):
index                       2415 non-null int64
username                    2415 non-null object
date                        2415 non-null object
retweets                    2415 non-null int64
favorites                   2415 non-null int64
text                        2415 non-null object
geo                         0 non-null float64
mentions                    402 non-null object
hashtags                    354 non-null object
id                          2415 non-null int64
permalink                   2415 non-null object
trump_18_tweet              2415 non-null object
timestamp                   2415 non-null datetime64[ns]
sentiment_vader_trump_18    2415 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(4), object(7)
memory usage: 264.2+ KB


In [30]:
from bokeh.io import show
from bokeh.layouts import gridplot, row ,column 
from bokeh.models import ColumnDataSource,ColorBar,LinearColorMapper,LogColorMapper
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc
from bokeh.models.widgets import Div


output_notebook()
# Make the ColumnDataSource: source
source = ColumnDataSource( data ={
    'x'       :  trump_18.date,
    'y'       : trump_18.sentiment_vader_trump_18,
    
})

# set min and max date times for plotting
xmin = trump_18['timestamp'].min() #- pd.Timedelta(days=0.1*N)
xmax = trump_18['timestamp'].max() #+ pd.Timedelta(days=0.1*N)



# Save the minimum and maximum values of the sentiment trump: ymin, ymax
ymin, ymax = -1 , 1

#add colors to plot
from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.models import (
    ColumnDataSource,
    ColorBar,
    LinearColorMapper,
    LogColorMapper,
)
from bokeh.palettes import Plasma256, RdBu

#maek a custome palette for plotting
custom= ['#b2182b','#d6604d','#f4a582','#fddbc7','#D9E2E0','#d1e4E0','#92c5de','#4393c3','#2166ac']
mapper = LinearColorMapper(palette=custom, low=-1, high=1)
mapper.high_color = 'blue'
mapper.low_color = 'red'
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

# Create the figure: plot
plot = figure( title = 'Trump Sentiment' , plot_height = 400 , plot_width = 700 , x_axis_type='datetime',
              x_range = (xmin, xmax), y_range = ( ymin , ymax ) )

# Add circle glyphs to the plot
plot.circle( trump_18.timestamp , trump_18['sentiment_vader_trump_18'] , fill_color={'field': 'y', 'transform': mapper}, line_color=None)

# Set the x-axis label
plot.xaxis.axis_label ='Date'

# Set the y-axis label
plot.yaxis.axis_label = 'sentiment vader trump 2018'

plot.add_layout(color_bar, 'right')
                     
# Add the plot to the current document and add a title
curdoc().add_root(plot)
curdoc().title = 'Sentiment (nltk:vader)'

show(plot)

Loading BokehJS ...

Cool. We have a pretty awesome picture of trump tweet sentiment over the last year. Bokeh allows you explore the graph interactively. Using the toolbar on the right. You can pan and zoom in. Play around with it. For example you can zoom in to just the last month. 

Observing the graph we see that his sentiment is spread out fairly consistly. No trends immediately jump out, so lets look at some basic statistics

In [31]:
trump_18.describe()

,index,retweets,favorites,geo,id,sentiment_vader_trump_18
count,2415.000000,2415.000000,2415.000000,0.0,2.415000e+03,2415.000000
mean,1207.000000,20228.929193,86679.066667,NaN,9.329462e+17,0.231237
std,697.294773,13463.426906,38477.252748,NaN,3.755039e+16,0.598796
min,0.000000,1155.000000,2738.000000,NaN,8.678010e+17,-0.974900
25%,603.500000,13223.000000,62406.000000,NaN,9.012841e+17,-0.273200
50%,1207.000000,17940.000000,80605.000000,NaN,9.282797e+17,0.419900
75%,1810.500000,24219.500000,103793.500000,NaN,9.659339e+17,0.764400
max,2414.000000,347592.000000,606598.000000,NaN,9.997616e+17,0.989900


 ## Tweet sentiment overall is actually positive! Interesting. 
 
Lets see how the news pull in some news tweets to see how they compare. We will also look at the sentiment of news tweets before trump tweets. The goal is to better understand a influencer, so lets see how the news tweets interact with trump.

We will look at CNN, FOX, NBC, and the New York Times. We will use the same scapper to get csv files for the last year. We will clean the data to keep only tweets with text as we did with Trump. We will also similiarly get the sentiment for each news tweet. Next we will call a function to collect all the tweets from the news broadcaster that occurred bewteen a trump tweet and the one before. The sum of the 


In [32]:
cnn_18 = pd.read_csv('cnn_18.csv')

In [33]:
cnn_18.dropna(axis=0,subset=['text'], inplace=True)#drop rows with NAN in text column

cnn_18.dropna(axis=0,how='all', inplace=True)#drop rows with NaN in all columns

cnn_18=cnn_18.reset_index()#reset index to replace dropped indices



In [52]:
clean_tweet_csv( cnn_18, 'cnn')# call function to clean text from scrapping add to cnn dataframe
sentiment_vader_tweets( cnn_18 , 'cnn_tweet' , 'cnn' )#call function to get vader sentiment and add to news dataframe
news_tweet( cnn_18, 'cnn', trump_18)#call function to get news tweets between trump tweets save as list in trump dataframe
tweet_before_sentiment( cnn_18 , "cnn", trump_18)#call function to add sentiment of all tweets between trump tweet



100%|██████████| 2413/2413 [00:02<00:00, 1001.55it/s]


In [84]:
cnn_18.head(3)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,Unnamed: 10,cnn_tweet,timestamp,sentiment_vader_cnn
0,0,CNN,2018-05-24 16:52,71,176,Donald Trump's 2020 campaign and the Republica...,NaN,NaN,NaN,999800339783090178,https://twitter.com/CNN/status/999800339783090178,NaN,Donald Trump's 2020 campaign and the Republica...,2018-05-24 16:52:00,0.8291
1,1,CNN,2018-05-24 16:51,44,121,The British government has opened an investiga...,NaN,NaN,NaN,999799994898108416,https://twitter.com/CNN/status/999799994898108416,NaN,The British government has opened an investiga...,2018-05-24 16:51:00,-0.4404
2,2,CNN,2018-05-24 16:46,53,105,Maduro's power grab can teach us how to defend...,NaN,@CNNOpinion,NaN,999798719317970946,https://twitter.com/CNN/status/999798719317970946,NaN,Maduro's power grab can teach us how to defend...,2018-05-24 16:46:00,0


In [85]:
trump_18.head(3)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,trump_18_tweet,timestamp,sentiment_vader_trump_18,cnn_text_before_tweet,nbc_text_before_tweet,cnn_sentiment_before_tweet
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,https://twitter.com/realDonaldTrump/status/999...,Today it was my great honor to present the Med...,2018-05-24 14:18:00,0.8316,[A University of Southern California faculty o...,[House Democrats are inviting Parkland student...,0.0213074
1,1,realDonaldTrump,2018-05-24 10:54,8221,35190,"Today, it was my honor to sign #S2155 , the Ec...",NaN,NaN,#S2155,999710267096354819,https://twitter.com/realDonaldTrump/status/999...,Today it was my honor to sign S2155 the Econo...,2018-05-24 10:54:00,0.8360,[10 to 16 named storms -- including five to ni...,[Democrat Schiff added to Justice Department b...,0.00828889
2,2,realDonaldTrump,2018-05-24 09:57,15597,58565,I have decided to terminate the planned Summit...,NaN,NaN,NaN,999695988813189120,https://twitter.com/realDonaldTrump/status/999...,I have decided to terminate the planned Summit...,2018-05-24 09:57:00,0.6249,[President Trump: NFL players who don't stand ...,[Bethany Mandel: Lil Tay isn't the first child...,-0.435933


## Lets get NBC

In [86]:
nbc_18 = pd.read_csv('nbc_18.csv')

In [87]:
nbc_18.dropna(axis=0,subset=['text'], inplace=True)#drop rows with NAN in text column

nbc_18.dropna(axis=0,how='all', inplace=True)#drop rows with NaN in all columns

nbc_18=nbc_18.reset_index()#reset index to replace dropped indices

In [88]:
clean_tweet_csv( nbc_18, 'nbc')# call function to clean text from scrapping add to cnn dataframe
sentiment_vader_tweets( nbc_18 , 'nbc_tweet' , 'nbc' )#call function to get vader sentiment and add to news dataframe
news_tweet( nbc_18, 'nbc', trump_18)#call function to get news tweets between trump tweets save as list in trump dataframe
tweet_before_sentiment( nbc_18 , "nbc", trump_18)#call function to add sentiment of all tweets between trump tweet

100%|██████████| 2413/2413 [00:02<00:00, 896.56it/s]


In [89]:
#inspect
nbc_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,Unnamed: 10,Unnamed: 11,nbc_tweet,timestamp,sentiment_vader_nbc
0,0,NBCNews,2018-05-24 16:45,89,84,A U.S. Border Patrol agent shot and killed a m...,NaN,NaN,NaN,9.997985e+17,https://twitter.com/NBCNews/status/99979848275...,NaN,NaN,A U.S. Border Patrol agent shot and killed a m...,2018-05-24 16:45:00,-0.6705


In [91]:
#inspect
trump_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,trump_18_tweet,timestamp,sentiment_vader_trump_18,cnn_text_before_tweet,nbc_text_before_tweet,cnn_sentiment_before_tweet,nbc_sentiment_before_tweet
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,https://twitter.com/realDonaldTrump/status/999...,Today it was my great honor to present the Med...,2018-05-24 14:18:00,0.8316,[A University of Southern California faculty o...,[House Democrats are inviting Parkland student...,0.0213074,0.114326


## Now lets get the New York Times

In [92]:
#load web scrapped tweets
nyt_18 = pd.read_csv('nyt_18.csv')

In [93]:
#drop rows with NAN in text column
nyt_18.dropna(axis=0,subset=['text'], inplace=True)
#drop rows with NaN in all columns
nyt_18.dropna(axis=0,how='all', inplace=True)
#reset index to replace dropped indices
nyt_18=nyt_18.reset_index()

In [94]:
clean_tweet_csv( nyt_18, 'nyt')# call function to clean text from scrapping add to cnn dataframe
sentiment_vader_tweets( nyt_18 , 'nyt_tweet' , 'nyt' )#call function to get vader sentiment and add to news dataframe
news_tweet( nyt_18, 'nyt', trump_18)#call function to get news tweets between trump tweets save as list in trump dataframe
tweet_before_sentiment( nyt_18 , "nyt", trump_18)#call function to add sentiment of all tweets between trump tweet

100%|██████████| 2413/2413 [00:02<00:00, 900.82it/s]


In [96]:
nyt_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,Unnamed: 10,nyt_tweet,timestamp,sentiment_vader_nyt
0,0,nytimes,2018-05-24 16:32,61,349,"David Sedaris loves audiences, audiences love ...",NaN,NaN,NaN,9.997952e+17,https://twitter.com/nytimes/status/99979519892...,NaN,David Sedaris loves audiences audiences love h...,2018-05-24 16:32:00,0.886


In [97]:
trump_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,trump_18_tweet,timestamp,sentiment_vader_trump_18,cnn_text_before_tweet,nbc_text_before_tweet,cnn_sentiment_before_tweet,nbc_sentiment_before_tweet,nyt_text_before_tweet,nyt_sentiment_before_tweet
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,https://twitter.com/realDonaldTrump/status/999...,Today it was my great honor to present the Med...,2018-05-24 14:18:00,0.8316,[A University of Southern California faculty o...,[House Democrats are inviting Parkland student...,0.0213074,0.114326,[André Leon Talley: I wish fashion was an easi...,-0.0182588


## Now lets get Fox News

In [98]:
fox_18 = pd.read_csv('fox_18.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (45,46,47,48,49,50,51,52,53,54,55,56,57,60,62,63,64,65,68,70,71,72,73,76,78,80,81,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [100]:
fox_18.dropna(axis=0,subset=['text'], inplace=True)#drop rows with NAN in text column

fox_18.dropna(axis=0,how='all', inplace=True)#drop rows with NaN in all columns

fox_18=fox_18.reset_index()#reset index to replace dropped indices

In [101]:
clean_tweet_csv( fox_18, 'fox')# call function to clean text from scrapping add to cnn dataframe
sentiment_vader_tweets( fox_18 , 'fox_tweet' , 'fox' )#call function to get vader sentiment and add to news dataframe
news_tweet( fox_18, 'fox', trump_18)#call function to get news tweets between trump tweets save as list in trump dataframe
tweet_before_sentiment( fox_18 , "fox", trump_18)#call function to add sentiment of all tweets between trump tweet


 15%|█▍        | 7683/51441 [00:15<01:28, 495.06it/s]


 28%|██▊       | 14589/51441 [00:31<01:18, 467.23it/s]


 40%|███▉      | 20384/51441 [00:46<01:11, 434.91it/s]


 51%|█████     | 26017/51441 [01:01<01:00, 422.78it/s]


 60%|█████▉    | 30716/51441 [01:16<00:51, 400.29it/s]


...

 83%|████████▎ | 42503/51441 [02:00<00:25, 353.05it/s]


 88%|████████▊ | 45414/51441 [02:16<00:18, 332.59it/s]


 93%|█████████▎| 48085/51441 [02:32<00:10, 315.19it/s]


 99%|█████████▊| 50763/51441 [02:48<00:02, 301.71it/s]


  0%|          | 0/2413 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide

100%|██████████| 2413/2413 [00:03<00:00, 755.34it/s]

In [102]:
#inspect
fox_18.head(1)

,index,username,date,"retwe,ets",favorites,text,geo,"mentio,ns","hash,tag,s",id,...,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,fox_tweet,timestamp,sentiment_vader_fox
0,0,FoxNews,"2018-05-24,""16:50",137,640,"Bishop Michael"", Curry ,on ,#RoyalWedding ,: ,...",NaN,NaN,"#RoyalWedding"",""#TheStory","999799945149509632""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bishop Michael Curry on RoyalWedding : I reall...,2018-05-24 16:50:00,0.9705


In [103]:
#inspect 
trump_18.head(1)

,index,username,date,retweets,favorites,text,geo,mentions,hashtags,id,...,timestamp,sentiment_vader_trump_18,cnn_text_before_tweet,nbc_text_before_tweet,cnn_sentiment_before_tweet,nbc_sentiment_before_tweet,nyt_text_before_tweet,nyt_sentiment_before_tweet,fox_text_before_tweet,fox_sentiment_before_tweet
0,0,realDonaldTrump,2018-05-24 14:18,14051,64210,"Today, it was my great honor to present the #M...",NaN,@USNavy @WhiteHouse,#MedalOfHonor,999761589476655105,...,2018-05-24 14:18:00,0.8316,[A University of Southern California faculty o...,[House Democrats are inviting Parkland student...,0.0213074,0.114326,[André Leon Talley: I wish fashion was an easi...,-0.0182588,[Marie Harf on NFL kneeling policy: It doesn't...,-0.00054


In [109]:
trump_18.fox_sentiment_before_tweet= trump_18.fox_sentiment_before_tweet.astype('float64')
trump_18.cnn_sentiment_before_tweet= trump_18.cnn_sentiment_before_tweet.astype('float64')
trump_18.nyt_sentiment_before_tweet= trump_18.nyt_sentiment_before_tweet.astype('float64')
trump_18.nbc_sentiment_before_tweet= trump_18.nbc_sentiment_before_tweet.astype('float64')

In [110]:
trump_18.describe()

,index,retweets,favorites,geo,id,sentiment_vader_trump_18,cnn_sentiment_before_tweet,nbc_sentiment_before_tweet,nyt_sentiment_before_tweet,fox_sentiment_before_tweet
count,2415.000000,2415.000000,2415.000000,0.0,2.415000e+03,2415.000000,2146.000000,1974.000000,2073.000000,1846.000000
mean,1207.000000,20228.929193,86679.066667,NaN,9.329462e+17,0.231237,-0.084623,-0.092030,-0.052120,-0.046573
std,697.294773,13463.426906,38477.252748,NaN,3.755039e+16,0.598796,0.242029,0.257940,0.246312,0.233220
min,0.000000,1155.000000,2738.000000,NaN,8.678010e+17,-0.974900,-0.939300,-0.959500,-0.956500,-0.911800
25%,603.500000,13223.000000,62406.000000,NaN,9.012841e+17,-0.273200,-0.178769,-0.198590,-0.148000,-0.132844
50%,1207.000000,17940.000000,80605.000000,NaN,9.282797e+17,0.419900,-0.067305,-0.082350,-0.033667,-0.029957
75%,1810.500000,24219.500000,103793.500000,NaN,9.659339e+17,0.764400,0.008474,0.008679,0.044368,0.049975
max,2414.000000,347592.000000,606598.000000,NaN,9.997616e+17,0.989900,0.927400,0.873300,0.851900,0.921100


## Now lets look at the News Sentiment over the same period

In [112]:
# plot news sentiment
#plot to noteboo
output_notebook()

#set y-axis scale
ymin, ymax = -1 , 1

# set min and max date times for plotting
xmin = trump_18['timestamp'].min() #- pd.Timedelta(days=0.1*N)
xmax = trump_18['timestamp'].max() #+ pd.Timedelta(days=0.1*N)

#make a custom color palette for plotting
custom= ['#b2182b','#d6604d','#f4a582','#fddbc7','#D9E2E0','#d1e4E0','#92c5de','#4393c3','#2166ac']
#set color bar scale values
mapper = LinearColorMapper(palette=custom, low=-1, high=1)
mapper.high_color = 'blue'
mapper.low_color = 'red'
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

#set x and y values for plotting
x = list(trump_18.timestamp)
y1 = list(trump_18.fox_sentiment_before_tweet)
y2 = list(trump_18.cnn_sentiment_before_tweet)
y3 = list(trump_18.nbc_sentiment_before_tweet)
y4 = list(trump_18.nyt_sentiment_before_tweet)

# create first plot: p1
p1 = figure(title= "Fox ", x_range = (xmin, xmax), y_range = ( ymin , ymax ),)
p1.circle(x, y1 , fill_color={'field': 'y', 'transform': mapper}, line_color=None, )
p1.xaxis.axis_label ='Date'
p1.yaxis.axis_label = 'vader sentiment'
p1.add_layout(color_bar, 'right')

#create plot p2:
p2 = figure(title= "CNN ",x_range = (xmin, xmax), y_range = ( ymin , ymax ) )
p2.circle(x, y2 , fill_color={'field': 'y', 'transform': mapper}, line_color=None,)
p2.xaxis.axis_label ='Date'
p2.yaxis.axis_label = 'vader sentiment'
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
p2.add_layout(color_bar, 'right')

#create plot p3
p3 = figure(title= "NBC ",x_range = (xmin, xmax), y_range = ( ymin , ymax ) )
p3.circle(x, y3, fill_color={'field': 'y', 'transform': mapper}, line_color=None,)
p3.xaxis.axis_label ='Date'
p3.yaxis.axis_label = 'vader sentiment'
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
p3.add_layout(color_bar, 'right')

#create plot p4
p4 = figure(title= "New York Times ",x_range = (xmin, xmax), y_range = ( ymin , ymax ) )
p4.circle(x, y4, fill_color={'field': 'y', 'transform': mapper}, line_color=None,)
p4.xaxis.axis_label ='Date'
p4.yaxis.axis_label = 'vader sentiment'
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
p4.add_layout(color_bar, 'right')

# make a grid
grid = gridplot([p1, p2, p3,p4], ncols=2, plot_width=400, plot_height=300)

#make a headline
show(column(Div(text='<h1>News Sentiment Before each Trump Tweet')))
# show the results
show(grid)

Loading BokehJS ...

In [114]:
#collect sentiment columns names into one variable
sentiment_columns=['sentiment_vader_trump_18','fox_sentiment_before_tweet','cnn_sentiment_before_tweet','nbc_sentiment_before_tweet','nyt_sentiment_before_tweet']

In [115]:
trump_18[sentiment_columns].corr()

,sentiment_vader_trump_18,fox_sentiment_before_tweet,cnn_sentiment_before_tweet,nbc_sentiment_before_tweet,nyt_sentiment_before_tweet
sentiment_vader_trump_18,1.000000,0.000538,0.066650,-0.001874,0.060123
fox_sentiment_before_tweet,0.000538,1.000000,0.119016,0.090878,0.001902
cnn_sentiment_before_tweet,0.066650,0.119016,1.000000,0.089326,0.047413
nbc_sentiment_before_tweet,-0.001874,0.090878,0.089326,1.000000,0.052057
nyt_sentiment_before_tweet,0.060123,0.001902,0.047413,0.052057,1.000000


The correlation of Trump's sentiment to the average of the News' tweets before is not strong. Maybe if we break the tweets down by topic we can get a better picture of Trump Tweets. 